<a href="https://colab.research.google.com/github/JacopoMangiavacchi/Swift-TensorFlow-Sample-Notebooks/blob/master/FizzBuzz_Swift_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import Foundation
import TensorFlow

In [0]:
func bitEncode(_ n: UInt, bits: Int = 10) -> [Float] {
    var array = [Float](repeating: 0, count: bits)
    for i in 0..<bits {
        array[bits - 1 - i] = Float(n >> i & 1)
    }
    return array
}

func labelEncode(_ n: UInt) -> [Float] {
    if n > 0 {
        if n % 15 == 0 {
            return [1, 0, 0, 0]
        }
      
        if n % 3 == 0 {
            return [0, 1, 0, 0]
        }
      
        if n % 5 == 0 {
            return [0, 0, 1, 0]
        }
    }
  
    return [0, 0, 0, 1]
}

In [0]:
//Create a Training dataset from 101 to 1024 (NB 1024 Greatest number for 10 bits) 
let start = 101
let end = 1024
var X = [[Float]]()
var Y = [[Float]]()

for i in start..<end {
    X.append(bitEncode(UInt(i)))
    Y.append(labelEncode(UInt(i)))
}

In [0]:
struct FizzBuzz: Layer {
    var l1: Dense<Float>
    var l2: Dense<Float>
    var l3: Dense<Float>


    init(bits: Int = 1) {
        l1 = Dense<Float>(inputSize: bits, outputSize: 128, activation: relu)
        l2 = Dense<Float>(inputSize: 128, outputSize: 256, activation: relu)
        l3 = Dense<Float>(inputSize: 256, outputSize: 4) // activation: softfamx
    }
    
    @differentiable
    func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
        var output = l1.applied(to: input, in: context)
        output = l2.applied(to: output, in: context)
        return l3.applied(to: output, in: context)
    }
}

In [5]:
let x = Tensor<Float>(shape: [Int32(end - start), 10], scalars: Array(X.joined()))
let y = Tensor<Float>(shape: [Int32(end - start), 4], scalars: Array(Y.joined()))

print(x.shape)
print(y.shape)

TensorShape(dimensions: [923, 10])
TensorShape(dimensions: [923, 4])


In [0]:
let batchSize: Int32 = 128
let trainingIterations = Int32(end - start) / batchSize

func minibatch<T>(_ x: Tensor<T>, batchIndex: Int32) -> Tensor<T> {
  let start = batchIndex * batchSize
  return x[start..<start+batchSize]
}

In [7]:
let optimizer = RMSProp<FizzBuzz, Float>(learningRate: 0.01)
var model = FizzBuzz(bits: 10)

let trainingContext = Context(learningPhase: .training)
for epoch in 1...200 {
    var totalLoss: Float = 0

    for i in 0..<trainingIterations {
        let xBatch = minibatch(x, batchIndex: i)
        let yBatch = minibatch(y, batchIndex: i)

        //let gradients = gradient(at: model) { m -> Tensor<Float> in
        let (loss, gradients) = valueWithGradient(at: model) { m -> Tensor<Float> in
            let logit = m.applied(to: xBatch, in: trainingContext)
            let batchLoss = softmaxCrossEntropy(logits: logit, labels: yBatch)
            totalLoss += batchLoss.scalarized()
            return batchLoss
        }

        //print("Batch \(i) loss: \(loss)")

        optimizer.update(&model.allDifferentiableVariables, along: gradients)
    }


    if epoch % 100 == 0 {
        print("Epoch \(epoch) totalLoss: \(totalLoss)")
    }
}

Epoch 100 totalLoss: 0.0073113805
Epoch 200 totalLoss: 0.0008063916


In [0]:
var X_test = [[Float]]()
var Y_true = [[Float]]()

// Test number from 1 to 101
for i in 1...100 {
    X_test.append(bitEncode(UInt(i)))
    Y_true.append(labelEncode(UInt(i)))
}

let xTest = Tensor<Float>(shape: [100, 10], scalars: Array(X_test.joined()))
let yTest = model.inferring(from: xTest)

In [9]:
var errors = 0
var correct = 0
let ySoft = softmax(yTest)
let yMax = ySoft.argmax(squeezingAxis: 1).array
let yArray = Array(ySoft.array)
let description = ["FizzBuzz", "Fizz    ", "Buzz    ", "        "]

for i in 0..<100 {
    let prediction = Int(yMax[i].description)!
    let groundTruth = Y_true[i].firstIndex(of: 1.0)!
    print(i+1, description[groundTruth], description[prediction], Y_true[i], yArray[i])
    if prediction == groundTruth {
        correct += 1
    }
    else {
        errors += 1
    }
}

1                   [0.0, 0.0, 0.0, 1.0] [1.2623381e-18, 2.1609827e-11, 2.429978e-07, 0.99999976]
2                   [0.0, 0.0, 0.0, 1.0] [4.7959694e-22, 1.0356868e-07, 2.0858555e-09, 0.9999999]
3 Fizz     Fizz     [0.0, 1.0, 0.0, 0.0] [3.4448574e-18, 0.9995146, 0.00048534336, 7.883036e-08]
4                   [0.0, 0.0, 0.0, 1.0] [8.18925e-19, 1.60493e-05, 1.120561e-08, 0.9999839]
5 Buzz              [0.0, 0.0, 1.0, 0.0] [4.0101704e-18, 3.6209986e-08, 0.09016733, 0.90983266]
6 Fizz     Fizz     [0.0, 1.0, 0.0, 0.0] [5.9540815e-16, 0.9996613, 7.1387624e-10, 0.00033869097]
7                   [0.0, 0.0, 0.0, 1.0] [3.7447588e-20, 2.3268811e-09, 2.5262638e-05, 0.9999747]
8                   [0.0, 0.0, 0.0, 1.0] [3.0719483e-14, 9.898624e-07, 1.0015415e-07, 0.9999989]
9 Fizz     Fizz     [0.0, 1.0, 0.0, 0.0] [2.975625e-10, 0.9999536, 2.3429167e-10, 4.6341716e-05]
10 Buzz     Buzz     [0.0, 0.0, 1.0, 0.0] [1.1571705e-19, 0.04609151, 0.95388186, 2.6648802e-05]
11                   [0.0, 0.0,

In [10]:
// accuracy = (correctly predicted class / total testing class) × 100%
print("Errors:" , errors, " Correct:", correct, "Accuracy: ", Float(correct) / Float(correct + errors))



Errors: 15  Correct: 85 Accuracy:  0.85
